# AirbyteLoader

>[Airbyte](https://github.com/airbytehq/airbyte) is a data integration platform for ELT pipelines from APIs, databases & files to warehouses & lakes. It has the largest catalog of ELT connectors to data warehouses and databases.

This covers how to load any source from Airbyte into LangChain documents

In [1]:
% pip install -qU langchain-airbyte

from langchain_airbyte import AirbyteLoader

In [1]:
%pip install --editable ~/langchain/oss-py/libs/partners/airbyte

Obtaining file:///Users/erickfriis/langchain/oss-py/libs/partners/airbyte
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached airbyte_cdk-0.58.9-py3-none-any.whl.metadata (11 kB)
  Using cached duckdb-0.9.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (760 bytes)
  Using cached google_cloud_bigquery-3.17.2-py2.py3-none-any.whl.metadata (8.8 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
  Using cached pandas-2.1.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached pyarrow-14.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 4.5 MB/s eta 0:00:00
  Using cached python_ulid-2.2.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached rich-13.7.0-py3-no

In [3]:
loader = AirbyteLoader(
    source="source-faker",
    stream="users",
    config={"count": 100},
)
docs = loader.load()

In [8]:
print(data[0].page_content[:500])

abilities: 
ability: 
name: blaze
url: https://pokeapi.co/api/v2/ability/66/

is_hidden: False
slot: 1


ability: 
name: solar-power
url: https://pokeapi.co/api/v2/ability/94/

is_hidden: True
slot: 3

base_experience: 267
forms: 
name: charizard
url: https://pokeapi.co/api/v2/pokemon-form/6/

game_indices: 
game_index: 180
version: 
name: red
url: https://pokeapi.co/api/v2/version/1/



game_index: 180
version: 
name: blue
url: https://pokeapi.co/api/v2/version/2/



game_index: 180
version: 
n
